In [11]:
import parameter
import argparse
import sys

def main(argv=None):
    if argv is None:
        argv = sys.argv[1:]
    
    parser = argparse.ArgumentParser()
    data = parameter.data
    for key, value in data.items():
        parser.add_argument(f'--{key}', default=value, help=f'{key} parameter')
    
    parser.add_argument("--detect_anomaly", action="store_true", default=False)
    parser.add_argument("--test_iterations", nargs="+", type=int, default=[100, 1_000, 7_000, 30_000])
    parser.add_argument("--save_iterations", nargs="+", type=int, default=[100, 1_000, 7_000, 30_000])    
    parser.add_argument("--checkpoint_iterations", nargs="+", type=int, default=[100, 1_000, 7_000, 30_000])
    parser.add_argument("--start_checkpoint", type=str, default=None)
    
    args = parser.parse_args(argv)
    
    
    for key, value in vars(args).items():
        print(f'{key}: {value}')

# 测试
argv = ['--detect_anomaly', '--test_iterations', '100', '2000', '--save_iterations', '100', '2000']

main(argv)


sh_degree: 3
source_path: 
model_path: 
images: images
resolution: -1
white_background: False
data_device: cuda
eval: False
iterations: 30000
position_lr_init: 0.00016
position_lr_final: 1.6e-06
position_lr_delay_mult: 0.01
position_lr_max_steps: 30000
feature_lr: 0.0025
opacity_lr: 0.05
scaling_lr: 0.005
rotation_lr: 0.001
percent_dense: 0.01
lambda_dssim: 0.2
densification_interval: 100
opacity_reset_interval: 3000
densify_from_iter: 500
densify_until_iter: 15000
densify_grad_threshold: 0.0002
convert_SHs_python: False
compute_cov3D_python: False
detect_anomaly: True
test_iterations: [100, 2000]
save_iterations: [100, 2000]
checkpoint_iterations: [100, 1000, 7000, 30000]
start_checkpoint: None
